# Deploy your model on a SageMaker Endpoint and visualize predictions

## Setup environment

In [ ]:
import cv2
import sagemaker
import numpy as np
import matplotlib.pyplot as plt
from sagemaker import get_execution_role
from sagemaker.utils import name_from_base
from sagemaker.tensorflow import TensorFlowModel
import visualization_utils as viz_utils

role = sagemaker.get_execution_role()
model_artefact = '<your-model-s3-path>'

## Create endpoint

In [ ]:
model = TensorFlowModel(
    name=name_from_base('tf2-object-detection'),
    model_data=model_artefact,
    role=role,
    framework_version='2.2'
)

In [ ]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')

## Predict

In [ ]:
def image_file_to_tensor(path):
    cv_img = cv2.imread(path,1).astype('uint8')
    cv_img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
    return cv_img

In [ ]:
img = image_file_to_tensor('test_images/22673445.jpg')

input = {
  'instances': [img.tolist()]
}

detections = predictor.predict(input)['predictions'][0]

img_with_detections = img.copy()
category_index = {1:{'id': 1, 'name': 'bee'}}
detection_boxes = np.array(detections['detection_boxes'])
detection_classes = [int(x) for x in detections['detection_classes']]
detection_scores = detections['detection_scores']

In [ ]:
%matplotlib inline

viz_utils.visualize_boxes_and_labels_on_image_array(
    img_with_detections,
    detection_boxes,
    detection_classes,
    detection_scores,
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=100,
    min_score_thresh=0.60,
    agnostic_mode=False)

plt.figure(figsize=(10,10))
plt.imshow(img_with_detections)
plt.show()

## Delete endpoint

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)